In [321]:
import pandas as pd
import numpy as np
import time 
import json

from pomegranate import State, DiscreteDistribution, HiddenMarkovModel
from sklearn.model_selection import train_test_split
from utils import load_saved_gfp_data, load_saved_mutated_gfp_data, one_hot_decode, one_hot_encode, normalize, count_substring_mismatch

In [322]:
class GenerativeHMM(): 
    
    def __init__(self, args, x_train=None, weights=None, verbose=True): 
        """
        Initializes the HMM to perform generative tasks
        Parameters
        ----------
        args : dictionary
            defines the hyper-parameters of the HMM
        args.name : string 
            defines the name of the HMM
        args.hidden_size : int 
            defines the hidden size
        args.input : tuple
            defines the shape of the inputs (should be 2 or 3 dimensional) 
        args.max_iterations: int
            sets the max iterations
        args.n_jobs: int
            sets the number of cores to use
        args.batch_size : int
            sets the batch size
        args.epochs : int 
            sets the epoch size 
        args.build_from_samples : boolean
            build model from samples
        """
        self.name = args["name"]
        self.hidden_size = args["hidden_size"]
        self.input = args["input"]
        self.max_iterations = args["max_iterations"]
        self.n_jobs = args["n_jobs"]
        self.batch_size = args["batch_size"]
        self.epoch = args["epoch"]
        if args["build_from_samples"] and x_train is not None: 
            self.model = HiddenMarkovModel.from_samples(DiscreteDistribution, 
                                                    n_components = self.hidden_size, 
                                                    X = x_train, 
                                                    algorithm = 'baum-welch', 
                                                    return_history = True,
                                                    verbose = verbose,
                                                    max_iterations = self.max_iterations,
                                                    n_jobs = self.n_jobs, 
                                                    weights = weights,
                                                    #batch_size = self.batch_size,
                                                    #batches_per_epoch = self.epochs
                                               )[0]
            
        else: 
            self.build_model()
        self.model.bake()

    
    def build_model(self): 
        base_pair_lst = ["A", "C", "T", "G"]
        dists = []
        for _ in range(self.hidden_size): 
            emission_probs = np.random.random(4)
            emission_probs = emission_probs / emission_probs.sum()
            dists.append(DiscreteDistribution(dict(zip(base_pair_lst, emission_probs))))
        trans_mat = np.random.random((self.hidden_size, self.hidden_size))
        trans_mat = trans_mat / trans_mat.sum(axis = 1, keepdims = 1)
        starts = np.random.random((self.hidden_size))
        starts = starts / starts.sum()
        # testing initializations
        np.testing.assert_almost_equal(starts.sum(), 1)
        np.testing.assert_array_almost_equal(np.ones(self.hidden_size), trans_mat.sum(axis = 1))
        self.model = HiddenMarkovModel.from_matrix(trans_mat, dists, starts)

    def fit(self, x_train, weights=None, verbose=True):
        """
        Assumes x_train is the same shape as the self.input shape. Fits the model on an HMM with self.hidden_size
        """    
        return self.model.fit(x_train, 
                        algorithm = 'baum-welch', 
                        return_history = True, 
                        verbose = verbose,
                        max_iterations = self.max_iterations,
                        n_jobs = self.n_jobs, 
                        weights = weights,
                        #batch_size = self.batch_size,
                        #batches_per_epoch = self.epochs
                   )
    
    def sample(self, n, length):
        """
        Input:
        n is number of samples
        length is how long you want each sample to be
        """
        return np.array(["".join(seq) for seq in self.model.sample(n = n, length = length)])
            
        
    def evaluate(self, x_test): 
        """
        evaluates the log probability of obtaining the sequences in x_test
        log(P(X1, X2, ..., X_test)) = sum(log(P(Xi)))
        Input: x_test a list of sequences. should be 2 or 3 dimensional
        """
        assert(len(np.array(x_test).shape) == 2 or len(np.array(x_test).shape) == 3)
        return sum([self.model.log_probability(seq) for seq in np.array(x_test)])
                
    def show_model(self): 
        self.model.plot()
        
    def save_model(self, path): 
        with open(path, 'w') as f:
            json.dump(self.model.to_json(), f)
    
    def load_model(self, path): 
        with open(path, 'r') as f:
            json_model = json.load(f)
        self.model = HiddenMarkovModel.from_json(json_model)
        

In [323]:
print("Loading data...")
start_time = time.time()
X_train, X_test, y_train, y_test = load_saved_gfp_data()
mutated_df = load_saved_mutated_gfp_data()
X_train_sequences = np.array([list(seq) for seq in one_hot_decode(X_train[0:100])])
print(X_train_sequences.shape)
print("Finished loading data in {0:.2f} seconds".format(time.time() - start_time))

Loading data...
(100, 714)
Finished loading data in 4.99 seconds


In [324]:
def hmm_base_args(): 
    return {
        "name" : "base HMM",
        "hidden_size" : 5,
        "input" : (100, 714),
        "max_iterations" : 10,
        "n_jobs" : 1,
        "batch_size" : 5,
        "epoch" : 2,
        "build_from_samples" : False
    }

def hmm_build_from_samples_args(): 
    args = hmm_base_args()
    args["build_from_samples"] = True
    return args

base_args = hmm_base_args()
build_from_samples_args = hmm_build_from_samples_args()

In [325]:
def test_fit_hmm(X_train_sequences): 
    args = hmm_base_args()
    hmm = GenerativeHMM(args)
    hmm.fit(X_train_sequences)
    args["max_iterations"] = 20
    args["n_jobs"] = 5
    hmm = GenerativeHMM(args)
    hmm.fit(X_train_sequences)
    
def test_fit_hmm_from_samples(X_train_sequences): 
    args = hmm_build_from_samples_args()
    hmm = GenerativeHMM(args, X_train_sequences)
    #test max iterations and n_jobs 
    args["max_iterations"] = 20
    args["n_jobs"] = 5
    hmm = GenerativeHMM(args, X_train_sequences)

def test_fit_hmm_weights(X_train_sequences):
    args = hmm_base_args()
    weights = np.identity(4)
    weights = np.vstack([weights, [0.25, 0.25, 0.25, 0.25]])
    for weight in weights:
        counts = {"A" : 0, "C" : 0, "T" : 0, "G" : 0}
        hmm = GenerativeHMM(args)
        hmm.fit(X_train_sequences, weight, verbose=False)
        json_model = json.loads(hmm.model.to_json())
        for state in json_model["states"]: 
            if state is not None and state["distribution"] is not None:
                mp = state["distribution"]["parameters"][0]
                for k, v in mp.items(): 
                    counts[k] = counts[k] + v
        print("Weights:", weight, "\nCounts:", counts)    
        

def test_fit_hmm_from_samples_weights(X_train_sequences):
    args = hmm_build_from_samples_args()
    weights = np.identity(4)
    weights = np.vstack([weights, [0.25, 0.25, 0.25, 0.25]])
    for weight in weights: 
        counts = {"A" : 0, "C" : 0, "T" : 0, "G" : 0}
        hmm = GenerativeHMM(args, X_train_sequences, weight, verbose=False)
        json_model = json.loads(hmm.model.to_json())
        for state in json_model["states"]: 
            if state is not None and state["distribution"] is not None:
                mp = state["distribution"]["parameters"][0]
                for k, v in mp.items(): 
                    counts[k] = counts[k] + v
        print("Weights:", weight, "\nCounts:", counts) 
        
def test_save_and_load_hmm(x_train_sequences): 
    args = hmm_base_args()
    hmm = GenerativeHMM(args)
    hmm.fit(X_train_sequences, verbose=False)
    hmm.save_model("./models/test.json")
    cached_hmm = GenerativeHMM(args)
    cached_hmm.load_model("./models/test.json")
    for i in "ACTG": 
        for j in "ACTG": 
            for k in "ACTG":
                codon = i + j + k
                np.testing.assert_almost_equal(hmm.evaluate([list(codon)]), cached_hmm.evaluate([list(codon)]))
    
    
def test_sample_and_evaluate_hmm(x_train_sequences): 
    args = hmm_base_args()
    hmm = GenerativeHMM(args)
    hmm.fit(x_train_sequences, verbose=False)
    seq1, seq2 = tuple(hmm.sample(2, 714))
    np.testing.assert_almost_equal(hmm.model.probability(seq1), np.e ** hmm.evaluate([list(seq1)]))
    total = 0
    for i in "ACTG": 
        for j in "ACTG": 
            for k in "ACTG":
                codon = i + j + k
                np.testing.assert_almost_equal(hmm.model.probability(codon), np.e ** hmm.evaluate([list(codon)]))
                total += np.e ** hmm.evaluate([list(codon)])
    np.testing.assert_almost_equal(total, 1)   

In [329]:
test_save_and_load_hmm(X_train_sequences)

In [326]:
test_sample_and_evaluate_hmm(X_train_sequences)

In [327]:
print("Test fit hmm")
synthetic_data = np.array([["A", "A", "A", "T"], ["C", "C", "C", "G"], ["T", "T", "T", "A"], ["G", "G", "G", "C"]])
test_fit_hmm_weights(synthetic_data)
print("\nTest fit hmm from samples weights:")
test_fit_hmm_from_samples_weights(synthetic_data)

Test fit hmm
Weights: [1. 0. 0. 0.] 
Counts: {'A': 3.1852188574167757, 'C': 0.0, 'T': 1.8147811425832239, 'G': 0.0}
Weights: [0. 1. 0. 0.] 
Counts: {'A': 0.0, 'C': 3.2129442322758064, 'T': 0.0, 'G': 1.7870557677241936}
Weights: [0. 0. 1. 0.] 
Counts: {'A': 1.3301250046205513, 'C': 0.0, 'T': 3.6698749953794487, 'G': 0.0}
Weights: [0. 0. 0. 1.] 
Counts: {'A': 0.0, 'C': 1.4471914475195748, 'T': 0.0, 'G': 3.5528085524804256}
Weights: [0.25 0.25 0.25 0.25] 
Counts: {'A': 0.8700817924709296, 'C': 1.5166766089955952, 'T': 1.1299182075290701, 'G': 1.4833233910044048}

Test fit hmm from samples weights:
Weights: [1. 0. 0. 0.] 
Counts: {'A': 3.569611728727529, 'C': 0.0, 'T': 1.4303882712724705, 'G': 0.0}
Weights: [0. 1. 0. 0.] 
Counts: {'A': 0.0, 'C': 3.1242679015408688, 'T': 0.0, 'G': 1.875732098459131}
Weights: [0. 0. 1. 0.] 
Counts: {'A': 2.067254058953376, 'C': 0.0, 'T': 2.932745941046624, 'G': 0.0}
Weights: [0. 0. 0. 1.] 
Counts: {'A': 0.0, 'C': 2.0858660777565254, 'T': 0.0, 'G': 2.91413392

In [328]:
test_fit_hmm(X_train_sequences)
test_fit_hmm_from_samples(X_train_sequences)

[1] Improvement: 3299.945057351637	Time (s): 0.2205
[2] Improvement: 17.351066011178773	Time (s): 0.2179
[3] Improvement: 6.922504063040833	Time (s): 0.2257
[4] Improvement: 4.748723868251545	Time (s): 0.2332
[5] Improvement: 4.185291902336758	Time (s): 0.2087
[6] Improvement: 3.9491171033587307	Time (s): 0.2258
[7] Improvement: 3.814449769197381	Time (s): 0.2134
[8] Improvement: 3.73763668758329	Time (s): 0.2272
[9] Improvement: 3.707303385453997	Time (s): 0.2248
[10] Improvement: 3.7189038851793157	Time (s): 0.2088
Total Training Improvement: 3352.0800540272176
Total Training Time (s): 2.4884
[1] Improvement: 12772.972851785787	Time (s): 0.08697
[2] Improvement: 69.98605718364706	Time (s): 0.09651
[3] Improvement: 59.39522110926919	Time (s): 0.08275
[4] Improvement: 54.511270776056335	Time (s): 0.1029
[5] Improvement: 54.175986628761166	Time (s): 0.07213
[6] Improvement: 56.829158752501826	Time (s): 0.08672
[7] Improvement: 61.81163960597769	Time (s): 0.1315
[8] Improvement: 69.12948